In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
df = pd.read_csv('sentiment_spanish.csv', header= None)

In [68]:
df.head()

,0,1
0,Acabo de tener un buen dia,1
1,Que increible es mi vida estoy muy feliz,1
2,La vida siempre te demuestra que hay razones p...,1
3,Me siento triste,0
4,Por favor matenme,0


In [69]:
df = df.rename(columns ={0:"Message", 1:"Value"}) #Name for columns
df.head()

,Message,Value
0,Acabo de tener un buen dia,1
1,Que increible es mi vida estoy muy feliz,1
2,La vida siempre te demuestra que hay razones p...,1
3,Me siento triste,0
4,Por favor matenme,0


In [70]:
df = df.dropna(axis = 0, how ='any')  #Droping possible NA

In [71]:
df.head()

,Message,Value
0,Acabo de tener un buen dia,1
1,Que increible es mi vida estoy muy feliz,1
2,La vida siempre te demuestra que hay razones p...,1
3,Me siento triste,0
4,Por favor matenme,0


In [72]:
df.shape

(183, 2)

In [73]:
df['Value'] = df.iloc[:,1].astype(int) #Changing floats to ints

In [74]:
df.head()
df.shape

(183, 2)

In [75]:
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import string

In [76]:
text_features = df['Message']
text_features = text_features.apply(lambda x: re.sub('[^a-z]', ' ', x.lower()))
#Taking only lowercases

In [77]:
print(text_features)

0                             acabo de tener un buen dia
1               que increible es mi vida estoy muy feliz
2      la vida siempre te demuestra que hay razones p...
3                                       me siento triste
4                                      por favor matenme
5                                        me vales madres
6                                             ya callate
7                                    eres lo que mas amo
8                                        yo no te quiero
9                       creo que la vida es muy aburrida
10                                      renovar es bueno
11                                           sal de aqui
12                                   es demasiado comodo
13                        te amo eres el amor de mi vida
14                               siempre amate como eres
15                                            no te amo 
16                                                baboso
17                             

In [78]:
#text_features= text_features.apply(lambda x: re.sub(' +', ' ', x))
#print(text_features)

In [79]:
text_features = text_features.apply(lambda x: ' '.join([word for word in x.split()
                                                  if word not in stopwords.words('spanish')]))
#Removing all unnnecesary words with stopwords


In [80]:
vectorizer = TfidfVectorizer("spanish") #Creating Object to classify phrases

In [81]:
features = vectorizer.fit_transform(text_features) #Transforming the data
features.shape

(183, 368)

In [82]:
X_train, X_test, y_train, y_test = train_test_split(features, df['Value'], test_size = 0.2, random_state=200)
X_train.shape
#Training and checking shapes

(146, 368)

In [83]:
X_test.shape

(37, 368)

In [84]:
y_train.shape

(146,)

In [85]:
y_test.shape

(37,)

In [86]:
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

'''Checking witch model is better for the classification'''

'Checking witch model is better for the classification'

In [87]:
svc = SVC(kernel='sigmoid', gamma=1.0)
knc = KNeighborsClassifier(n_neighbors=2)
mnb = MultinomialNB(alpha=0.2)
dtc = DecisionTreeClassifier(min_samples_split=7)
rfc = RandomForestClassifier(n_estimators=31)

In [88]:
mnb.fit(X_train, y_train)
svc.fit(X_train, y_train)
knc.fit(X_train, y_train)
dtc.fit(X_train, y_train)
rfc.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=31, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [89]:
from sklearn.metrics import r2_score
#Checking its scores

In [90]:
print(mnb.score(X_test,y_test))

0.72972972973


In [91]:
print(knc.score(X_test,y_test))

0.567567567568


In [92]:
print(svc.score(X_test,y_test))

0.675675675676


In [93]:
print(dtc.score(X_test,y_test))

0.567567567568


In [94]:
print(rfc.score(X_test,y_test))

0.513513513514


In [112]:
test = #Frase
delete = re.compile('[^a-zA-Z]')
test = delete.sub(' ', test)
test = test.lower()
test= ' '.join([word for word in test.split()
                   if word not in stopwords.words('spanish')])
test
#Applying the prediction

'voluntad ayudo triste'

In [96]:
tfidf_svm = Pipeline([('tfidf', vectorizer), ('mnb', mnb)])
#Joining the Tfidf object model and the SVC MODEL 

In [97]:
from sklearn.externals import joblib
joblib.dump(tfidf_svm, 'model_spanish.pkl', protocol = 2)
#Creating the file of the model

['model_spanish.pkl']

In [98]:
loaded_model = joblib.load('model_spanish.pkl') #Loading it

In [113]:
#testing and getting a correct answer
result = loaded_model.predict([test])
print("X=%s, Predicted=%s" % (test[0], result[0]))

X=v, Predicted=0
